In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import numpy as np
import pandas as pd
import os
from scipy.io import loadmat

In [40]:
def _pick_label_key(keys):
    candidates = ['labels', 'label', 'gt', 'gnd', 'Y', 'truth', 'truelabel','y']
    for c in candidates:
        if c in keys:
            return c
    return None

def load_multiview_mat(mat_path, verbose=True, save_csv=True, out_dir=None):
    views, labels = [], None

    mat = loadmat(mat_path)
    keys = [k for k in mat.keys() if not k.startswith("__")]
    if verbose: print("[scipy] keys:", keys)

    # Find a container for multi-view data
    data_key = None
    for cand in ['data', 'views', 'X', 'fea']:
        if cand in keys:
            data_key = cand
            break

    if data_key is not None:
        container = mat[data_key]
        # Case A: cell array -> numpy object array
        if isinstance(container, np.ndarray) and container.dtype == object:
            cells = container.ravel().tolist()
            first = np.array(cells[0])
            n = max(first.shape)
            for c in cells:
                arr = np.array(c)
                views.append(arr)

        else:
            raise ValueError(f"Unsupported container type for '{data_key}'.")

    # Labels
    lab_key = _pick_label_key(keys)
    if lab_key is not None:
        labels = np.array(mat[lab_key]).squeeze().astype(int)

    # Verbose summary
    if verbose:
        for i, X in enumerate(views):
            print(f"  view{i+1} shape: {X.shape}")
        if labels is not None:
            print(f"  labels shape: {labels.shape}")

    # SAVE CSV FILES
    if save_csv:
        if out_dir is None:
            out_dir = os.path.dirname(mat_path)
        os.makedirs(out_dir, exist_ok=True)

        for i, X in enumerate(views):
            df = pd.DataFrame(X)
            if labels is not None:
                df["label"] = labels  # append label column
            csv_path = os.path.join(out_dir, f"ProteinFoldld_view{i+1}.csv")
            df.to_csv(csv_path, index=False)
            print(f"Saved: {csv_path}")

        # Save labels separately too
        if labels is not None:
            labels_df = pd.DataFrame(labels, columns=["label"])
            labels_path = os.path.join(out_dir, "ProteinFold_labels.csv")
            labels_df.to_csv(labels_path, index=False)
            print(f"Saved: {labels_path}")

    return views, labels




In [41]:
views, labels = load_multiview_mat('/content/ProteinFold.mat', verbose=True, save_csv=True)

[scipy] keys: ['X', 'y']
  view1 shape: (694, 27)
  view2 shape: (694, 27)
  view3 shape: (694, 27)
  view4 shape: (694, 27)
  view5 shape: (694, 27)
  view6 shape: (694, 27)
  view7 shape: (694, 27)
  view8 shape: (694, 27)
  view9 shape: (694, 27)
  view10 shape: (694, 27)
  view11 shape: (694, 27)
  view12 shape: (694, 27)
  labels shape: (694,)
Saved: /content/ProteinFoldld_view1.csv
Saved: /content/ProteinFoldld_view2.csv
Saved: /content/ProteinFoldld_view3.csv
Saved: /content/ProteinFoldld_view4.csv
Saved: /content/ProteinFoldld_view5.csv
Saved: /content/ProteinFoldld_view6.csv
Saved: /content/ProteinFoldld_view7.csv
Saved: /content/ProteinFoldld_view8.csv
Saved: /content/ProteinFoldld_view9.csv
Saved: /content/ProteinFoldld_view10.csv
Saved: /content/ProteinFoldld_view11.csv
Saved: /content/ProteinFoldld_view12.csv
Saved: /content/ProteinFold_labels.csv
